In [1]:
import torch
from torch import nn
import pdb
from torch.nn import functional as F
from torch.utils.data import DataLoader as DataLoader
import dlc_practical_prologue as prologue

In [2]:
train_input, train_target, train_classes,test_input, test_target, test_classes= prologue.generate_pair_sets(1000)

In [3]:
train_classes = train_classes.t().reshape(2000,1)
test_classes = test_classes.t().reshape(2000,1)

In [4]:
def to_one_hot(classes):
    n, k = classes.size()
    one_hot_classes = torch.zeros(n,10)
    one_hot_classes[range(0,n),classes[:,0]] = 1
    return one_hot_classes

In [5]:
train_input= torch.cat([train_input[:,0,:,:],train_input[:,1,:,:]],0).view(2000,1,14,14)
test_input= torch.cat([test_input[:,0,:,:],test_input[:,1,:,:]],0).view(2000,1,14,14)

In [6]:
train_classes_oh, test_classes_oh = to_one_hot(train_classes),to_one_hot(test_classes)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # convolutional network
        self.conv1 = nn.Sequential(nn.Conv2d(1, 16, 3, stride=1, padding=0, dilation=1),
                                   nn.Tanh(),
                                   nn.MaxPool2d(2,stride=1),
                                   nn.Dropout(0.2),

                                   nn.Conv2d(16,64,3),
                                   nn.Tanh(),
                                   nn.MaxPool2d(2, stride=1),
                                   nn.Dropout(0.2),
        )

        # fully connected layer
        self.fc1 = nn.Sequential(nn.Linear(64*8*8,128),
                                 nn.Tanh(),
                                 nn.Linear(128,10),
                                 nn.Softmax(-1)
        )

        self.out = nn.Sequential(nn.Linear(10,1),nn.Softmax(-1))
        
    def forward(self, inputdata, aux_loss = True):
        n_epoch,dim, _, _ = inputdata.size()
        output = []
        data = inputdata
        output1 = self.conv1(data)
        output1 = output1.view(n_epoch,-1)
        output1 = self.fc1(output1)
#         output1 = self.out(output1)

        return output1
        

In [25]:
def run(data,label,n_epoch=50, train=True, test= False,model = CNN()):
    
    if train:
        learning_rate = 1e-3
        model.train()
        batch_size = 100
        loss_fn = nn.BCELoss()
        
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))
        ExpLR = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
        losses = []
        for index in range(n_epoch):
            total_loss = 0
            for batch_data, batch_label in zip(data.split(batch_size), label.split(batch_size)):
                model.train()
                pred = model(batch_data)

                loss =  loss_fn(pred, batch_label)
                
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                with torch.no_grad():
                    total_loss += loss
            
            losses.append(total_loss/20)
            
            pred = model(data)
            _,pred_label = torch.max(pred,1)
            _,true_label = torch.max(label,1)
            
            pred_label = pred_label.reshape(2,1000)
            _,pred_target = torch.max(pred_label,0)

            true_label = true_label.reshape(2,1000)
            _,true_target = torch.max(true_label,0)
            if index%5==0:
                print('loss of',index,'epoch is ',loss)
                print('accurancy of classification in training process-----',\
                  torch.sum(pred_label.type(torch.float)==true_label.type(torch.float))/2000.0)
            
                print('accurancy in training process-----',\
                  torch.sum(pred_target.type(torch.float)==true_target.type(torch.float))/1000.0)

        
    if test:
        loss_fn = nn.BCELoss()
        
        pred = model(data)
        losses =  loss_fn(pred, label)
        _,pred_label = torch.max(pred,1)
        _,true_label = torch.max(label,1)

        print('accurancy of classification process-----',torch.sum(pred_label.type(torch.float)==true_label.type(torch.float))/2000.0)
        pred_label = pred_label.reshape(2,1000)
        _,pred_target = torch.max(pred_label,0)
        
        true_label = true_label.reshape(2,1000)
        _,true_target = torch.max(true_label,0)
        print('accurancy in test process-----',\
              torch.sum(pred_target.type(torch.float)==true_target.type(torch.float))/1000.0)
    
    return losses,pred_label

In [26]:
losses,pred =  run(train_input,train_classes_oh, train=True, test= False)

loss of 0 epoch is  tensor(0.0957, grad_fn=<BinaryCrossEntropyBackward>)
accurancy of classification in training process----- tensor(0.8865)
accurancy in training process----- tensor(0.9120)
loss of 5 epoch is  tensor(0.0169, grad_fn=<BinaryCrossEntropyBackward>)
accurancy of classification in training process----- tensor(0.9845)
accurancy in training process----- tensor(0.9880)
loss of 10 epoch is  tensor(0.0074, grad_fn=<BinaryCrossEntropyBackward>)
accurancy of classification in training process----- tensor(0.9965)
accurancy in training process----- tensor(0.9980)
loss of 15 epoch is  tensor(0.0042, grad_fn=<BinaryCrossEntropyBackward>)
accurancy of classification in training process----- tensor(0.9910)
accurancy in training process----- tensor(0.9940)
loss of 20 epoch is  tensor(0.0016, grad_fn=<BinaryCrossEntropyBackward>)
accurancy of classification in training process----- tensor(1.)
accurancy in training process----- tensor(1.)
loss of 25 epoch is  tensor(0.0020, grad_fn=<Binar

In [27]:
_,pred =  run(test_input,test_classes_oh, train=False, test= True)

accurancy of classification process----- tensor(0.9495)
accurancy in test process----- tensor(0.9680)
